export cells as modules?



In [ ]:
var fs = require('fs');
var path = require('path');
var rimraf = require('rimraf');
var importer = require('../Core');
var mkdirpSync = importer.import('mkdirp');
var getExtension = importer.import('cell extension')

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var MODULE_OUTPUT = __dirname + '/.modules';
if(fs.existsSync(MODULE_OUTPUT)) {
    rimraf.sync(MODULE_OUTPUT);
}

function exportNotebook(notebook, project = __dirname + '/..') {
    const notebooks = typeof notebook === 'string' ? [notebook] : notebook;
    return notebooks.reduce((cells, notebook) => {
        const name = path.basename(notebook).replace(/\.ipynb/ig, '');
        const parent = path.resolve(path.dirname(notebook)).replace(path.resolve(project), '');
        mkdirpSync(path.join(MODULE_OUTPUT, parent, name));

        return importer.getCells(notebook, ['*', 'code', 'markdown', 'raw'])
            .reduce((results, cell, i) => {
                const extension = getExtension(cell, notebook);
                // TODO: get previous markdown and extract name that leads back to the current cell
                const cellPath = path.join(MODULE_OUTPUT, parent, name, 'cell-' + i + extension);
                fs.writeFileSync(cellPath, cell.source.join(''));
                results.push(cellPath);
                return results;
            }, cells);
    }, []);
}
module.exports = exportNotebook;


get cell extension?


In [ ]:

function getExtension(cell, notebook) {
    var extension;
    if(cell.cell_type === 'markdown') {
        extension = '.md';
    } else if(cell.cell_type === 'raw') {
        extension = '.txt';
    } else if(cell.language === 'javascript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)) {
            extension = '.spec.js'
        } else {
            extension = '.js';
        }
    } else if(cell.language === 'powershell') {
        extension = '.ps1';
    } else if(cell.language === 'csharp') {
        extension = '.cs';
    } else if(cell.language === 'python') {
        extension = '.py';
    } else if(cell.language === 'typescript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)) {
            extension = '.spec.ts'
        } else {
            extension = '.ts';
        }
    } else if(cell.language === 'bash') {
        extension = '.sh';
    } else {
        throw 'unknown language or cell type: ' + (cell.filename || notebook) + ' (' + cell.cell_type + ',' + cell.language + ')';
    }
    return extension;
}
module.exports = getExtension;


export all notebooks?

Export all notebooks to a structured folder where each cell has it's own file for linting?



In [ ]:
var glob = require('glob');
var path = require('path');
var importer = require('../Core');
var exportNotebook = importer.import('notebook.ipynb[export cells modules]');

function exportAll(project) {
    const notebooks = glob.sync('**/*.ipynb', {cwd: project});
    return exportNotebook(notebooks.map(n => path.join(project, n)), project);
}
module.exports = exportAll;


test notebook export?



In [ ]:
var path = require('path');
var glob = require('glob');
var assert = require('assert');
var fs = require('fs');
var importer = require('../Core');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = path.resolve(__dirname + '/../');
var MODULE_OUTPUT = PROJECT_PATH + '/Utilities/.modules';

var sortNumeric = (that) => {
    return that.sort((a, b) => parseInt(a.split(/[^0-9]/ig).join('')) - parseInt(b.split(/[^0-9]/ig).join('')));
}

describe('notebook export service', () => {
    it('should export this file', () => {
        exportNotebook(path.resolve(PROJECT_PATH + '/Utilities/notebook.ipynb'));
        var files = glob.sync('Utilities/notebook/*', {cwd: MODULE_OUTPUT});
        sortNumeric(files);
        assert(path.basename(files[0]) === 'cell-0.md');
        assert(path.basename(files[1]) === 'cell-1.js');
        assert(path.basename(files[2]) === 'cell-2.md');
    })
    
    it('should have comparable cells', () => {
        var importCells = importer.interpret(['notebook.ipynb[0]', 'notebook.ipynb[1]', 'notebook.ipynb[2]']);
        exportNotebook(path.resolve(PROJECT_PATH + '/Utilities/notebook.ipynb'));
        var files = glob.sync('Utilities/notebook/*', {cwd: MODULE_OUTPUT});
        sortNumeric(files);
        var fsCell0 = fs.readFileSync(path.join(MODULE_OUTPUT, files[0])).toString();
        var fsCell1 = fs.readFileSync(path.join(MODULE_OUTPUT, files[1])).toString();
        var fsCell2 = fs.readFileSync(path.join(MODULE_OUTPUT, files[2])).toString();
        assert(fsCell0 === importCells[0].markdown.join(''));
        assert(fsCell1 === importCells[0].fresher, fsCell1.length + ' != ' + importCells[0].fresher.length)
        assert(fsCell2 === importCells[1].markdown.join(''))
    })
    
    it('should export all notebooks', () => {
    //    var exported = exportAll(PROJECT_PATH);
    //    assert(exported.length > 0);
    })
})


import all cell modules?

import files back in to cells? (two-way workflow)


In [ ]:
var fs = require('fs');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Documents/jupytangular2/Utilities/.modules';
var project = PROFILE_PATH + '/Documents/jupytangular2';

var cells = glob.sync('**/cell-*', {cwd: PROJECT_PATH});
for(const c of cells) {
    const cell = path.basename(c);
    const notebook = path.basename(path.dirname(c));
    const parent = path.basename(path.dirname(path.dirname(c)));
    if(parent === 'jupytangular2') {
        continue;
    }
    const nb = JSON.parse(fs.readFileSync(path.join(project, parent, notebook + '.ipynb')));
    let counter = 0;
    for(const i in nb.cells) {
        if(!nb.cells.hasOwnProperty(i)) {
            continue;
        }
        // TODO: reimport all cells
        if(nb.cells[i].cell_type === 'code') {
            if(cell === 'cell-' + counter + '.js') {
                nb.cells[i].source = fs.readFileSync(path.join(PROJECT_PATH, c)).toString().split('\n');
            }
            counter++;
        }
    }
    fs.writeFileSync(path.join(project, parent, notebook + '.ipynb'), JSON.stringify(nb, null, 2));
}


In [ ]:
// TODO: delete .modules if everything checks out

// TODO: import tests


TODO:

import module as notebook cell?

import gist as notebook

import random instructions as notebook

rules:
- testing cells have zero markdown before them,
- all public functions are made public using module.export or global namespace
- use describe( test blocks to describe parsing and replacement information for the functions it is testing, i.e. function utility(root) would have a describe block describe('utility(root)') neatly formated and parsed for calendar commands
- every file is one feature
- every cell or export should be a single purpose component with proper includes/dependencies
- every cell must be under 100 lines
- every function must be testable insolation
- every function should be made accesible from the command line using module.exports
- every function should be runnable from notebooks using typeof $$ !== 'undefined'


```javascript

if(typeof $$ !== 'undefined') {
    $$.async();
    exportAndDeploy('../Frameworks/zuora to eloqua.ipynb')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}

```
- every module should have a markdown title, at least one question that the code intends to answer (how to?), includes at least (where name is the name of any function in the code block):


```javascript

...
function <name> () {
...
}
...
exports = <name>
...

```




utility function to export notebooks and deploy to lambda

In [1]:
// assuming you've already run `aws configure`
var fs = require('fs');
var os = require('os');
var path = require('path');
var importer = require('../Core');
var {
    execCmd,
    delint,
    exportNotebook,
    getExtension
} = importer.import([
    'spawn child process',
    'delinting notebooks',
    'export cells as modules',
    'cell extension'
]);

function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i])) ;
    return co;
};

function getImportsRecursively(cell, exports) {
    var newCell = cell
        .replace(/require\('\.\.\/Core'\).*\s*/igm, 'importer.import(\'import notebook.ipynb\');\n')
        .replace(/.*typeof \$\$[\s\S]*?\}/igm, '')
        .replace(/\bvar\b/igm, 'let');
    
    var imports = regexToArray(/(importNotebook|import)\(((\s*([\['"\{])\s*([\s\S]*?)\s*\2[,\s]*)+)\)/igm, newCell, 2);
    imports = imports.reduce((acc, i) => {
        return acc.concat(regexToArray(/([\['"\{])\s*([\s\S]*?)\s*\1/igm, i, 2));
    }, []);
    console.log(imports);
    imports.forEach(i => {
        const result = importer.interpret(i);
        if(result.length) {
            const names = result.reduce((acc, result) => {
                if(result.questions.length === 0) {
                    return acc;
                }
                result.name = niceName(result);
                if(typeof exports[result.name] === 'undefined') {
                    cellsFilenames(result.filename);
                    exports[result.name] = result;
                    exports[result.name].exportedCode = getImportsRecursively(result.code, exports);
                }
                acc.push(result.name);
                return acc;
            }, []);
            newCell = newCell.replace(
                new RegExp('(importNotebook|importer\.import)\\([\\[\'"]' + i + '[\'"\\]]\\)', 'igm'),
                '[require(\'./' + names.join('\'), require(\'./') + `')].reduce((acc, func, j) => {
if(typeof func === 'function') {
    acc[func.name] = func;
}
if(typeof func === 'object' && func !== null
    && typeof func[Object.keys(func)[0]] === 'function') {
    acc[func[Object.keys(func)[0]].name] = func[Object.keys(func)[0]];
}
acc[j] = func;
return acc;
}, {})`.replace(/\n/ig, ''));
        } else {
            result.name = niceName(result);
            if(typeof exports[result.name] === 'undefined') {
                cellsFilenames(result.filename);
                exports[result.name] = result;
                exports[result.name].exportedCode = getImportsRecursively(result.code, exports);
            }
            newCell = newCell.replace(
                new RegExp('(importNotebook|importer\.import)\\([\\[\'"]' + i + '[\'"\\]]\\)', 'igm'),
                'require(\'./' + result.name + '\')');
        }
    })
    return newCell;
}

// TODO: get previous markdown and extract name that leads back to the current cell
function niceName(cell) {
    cell.questions.sort((a, b) => a.length - b.length);
    return cell.questions[0].replace('?', '').replace(/[^a-z0-9]+|\btest\b/igm, ' ').trim().replace(/\s+/igm, '-') + getExtension(cell);
}

function cellsFilenames(notebook) {
    const entryCells = importer.interpret(path.basename(notebook));
    const exports = exportNotebook(notebook, '..');
    return entryCells.reduce((acc, cell) => {
        if(cell.questions.length > 0) {
            const name = niceName(cell);
            acc[name] = cell;
            cell.name = name;
        }
        return acc;
    }, {});
}

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var PROJECT_PATH = path.join(__dirname, '../.output');
if(!fs.existsSync(PROJECT_PATH)) {
    fs.mkdirSync(PROJECT_PATH);
}

function exportAndDeploy(notebook) {
    /*if(path.basename(notebook) === 'import notebook.ipynb') {
        console.log('emitting import-notebook.js');
        const imports = importer.interpret('import notebook.ipynb');
        fs.writeFileSync(path.join(PROJECT_PATH, 'import-notebook.js'), imports.map(cell => cell.code).join(''));
        return Promise.resolve(imports);
    }*/
    // these are the top level cells starting the import tree
    const entryCells = cellsFilenames(notebook);
    const imports = Object.keys(entryCells).reduce((acc, k) => {
        acc[k].exportedCode = getImportsRecursively(entryCells[k].code, acc);
        return acc;
    }, entryCells);
    Object.keys(imports).forEach(e => {
        console.log('emitting ' + e);
        fs.writeFileSync(path.join(PROJECT_PATH, e), imports[e].exportedCode);
    });
    // TODO: zip and upload to AWS
    return Promise.resolve(imports);
    //return delint(PROJECT_PATH)
}
module.exports = exportAndDeploy;

if(typeof $$ !== 'undefined') {
    $$.async();
    exportAndDeploy('../Core/import notebook.ipynb')
        .then(() => exportAndDeploy('../Frameworks/rpc.ipynb'))
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


[]
[]
[]
[]
[ 'interpret all notebooks.ipynb' ]
[]
[ 'import notebook.ipynb', 'interpret questions.ipynb' ]
[]
[ 'import notebook.ipynb' ]
[]
[]
[]
[ 'import notebook.ipynb', 'interpret questions.ipynb' ]
[]
[]
emitting import-notebook.js
emitting stream-json-using-a-match-function.js
emitting parse-cells-of-types-from-a-notebook.js
emitting run-all-promises-sequentially.js
emitting search-notebook-questions.js
emitting How-does-node-module-require-work.js
emitting Import-arbitrary-code-in-to-a-new-context.js
emitting search-all-directories.js
emitting find-all-notebooks.js
emitting turn-jupyter-notebooks-in-to-modules.js
emitting cache-notebook-questions-in-fuse.js
emitting read-markdown-leading-up-to-code-cells.js
emitting find-functions-in-jupyter-notebooks.js
emitting display-interpreted-results-in-markdown.js
emitting notebook-search.spec.js
[ 'import notebook.ipynb' ]
[]
[]
[]
[]
[ 'interpret all notebooks.ipynb' ]
[]
[ 'import notebook.ipynb', 'interpret questions.ipynb' ]
[]
[]

{ 'rpc-permissions.js': 
   { code: 'var importer = require(\'../Core\');\nvar FUNCTION_GROUPS = [\'Function\', \'Public\'];\nvar SELENIUM_GROUPS = [\'Selenium\'];\nvar UNITTEST_GROUPS = [\'Unit Test\']; // TODO: identified using describe\nvar DEFAULT_GROUPS = []\nvar PUBLIC = {\n    \'google contacts.ipynb[google contact settings]\': [\'Function\', \'Inner Circle\', \'Public\'],\n    \'import notebook.ipynb[search notebook questions]\' : [\'Function\', \'Public\'],\n    \'rpc.ipynb[permissions]\' : [\'Function\', \'Public\'],\n    \'rpc.ipynb[parameter names]\' : [\'Function\', \'Public\'],\n    \'gulp.ipynb[search notebooks]\' : [\'Function\', \'Public\'],\n    \'data collection.ipynb[meta search all]\' : [\'Function\', \'Public\'],\n    \'data collection.ipynb[schedule search all]\' : [\'Function\', \'Public\'],\n    \'data collection.ipynb[tell joke]\' : [\'Function\', \'Public\'],\n};\nvar REGEX_EXPORTS = /function\\s*([^\\(]*)\\s*\\([\\s\\S]*?exports[^=]*?\\s*=\\s*\\1\\s*;/ig;\nv